In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML, display
pd.set_option('display.max_columns', None)

In [2]:
#Weekly Stats

#Week 1
Week_1_Stats = pd.read_csv('Week_1_NFL_2025.csv', quotechar="'")
Week_1_Stats.columns = Week_1_Stats.columns.str.replace('"', '', regex=False)
Week_1_Stats['Rk'] = Week_1_Stats['Rk'].str.replace('"', '', regex=False)
Week_1_Stats = Week_1_Stats.drop(['Rk', 'Day', 'Date', 'Unnamed: 12', 'Opp', 'Result', 'Att', 'Att.1', 'Tgt', 'G#', 'Week', 'OffSnp'], axis=1)
Week_1_Stats = Week_1_Stats.rename(columns={'1D': 'Rush1D', '1D.1': 'Rec1D', 'OffSnp.1': 'OffSnp', 'Att.2': 'PassAtt','TD': 'PassTD', 'Yds': 'PassYds', 'Y/A': 'PassY/A', 'Yds.1': 'SackYds', 'Succ%': 'PassSucc%', 'Att.3': 'RushAtt','TD.1': 'RushTD', 'Yds.2': 'RushYds', 'Y/A/1': 'RushY/A', 'Succ%.1': 'RushSucc%', 'Tgt.1': 'Tgt', 'Yds.3': 'RecYds', 'TD.2': 'RecTD', 'Succ%.2': 'RecSucc%'})

#Week 2
Week_2_Stats = pd.read_csv('Week_2_NFL_2025.csv', quotechar="'")
Week_2_Stats.columns = Week_2_Stats.columns.str.replace('"', '', regex=False)
Week_2_Stats['Rk'] = Week_2_Stats['Rk'].str.replace('"', '', regex=False)
Week_2_Stats = Week_2_Stats.drop(['Rk', 'Day', 'Date', 'Unnamed: 12', 'Opp', 'Result', 'Att', 'Att.1', 'Tgt', 'G#', 'Week','OffSnp'], axis=1)
Week_2_Stats = Week_2_Stats.rename(columns={'1D': 'Rush1D', '1D.1': 'Rec1D', 'OffSnp.1': 'OffSnp', 'Att.2': 'PassAtt','TD': 'PassTD', 'Yds': 'PassYds', 'Y/A': 'PassY/A', 'Yds.1': 'SackYds', 'Succ%': 'PassSucc%', 'Att.3': 'RushAtt','TD.1': 'RushTD', 'Yds.2': 'RushYds', 'Y/A/1': 'RushY/A', 'Succ%.1': 'RushSucc%', 'Tgt.1': 'Tgt', 'Yds.3': 'RecYds', 'TD.2': 'RecTD', 'Succ%.2': 'RecSucc%'})

#print(Week_1_Stats.columns)

In [3]:
#NFL Schedule
Schedule = pd.read_csv('Schedule_2025.csv')
Schedule = Schedule.map(lambda x: x.replace('@', '') if isinstance(x, str) else x)

#Conform to Stathead Labels
Schedule = Schedule.map(lambda x: x.replace('GB', 'GNB') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('KC', 'KAN') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('LV', 'LVR') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('NO', 'NOR') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('NE', 'NWE') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('SF', 'SFO') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('TB', 'TAM') if isinstance(x, str) else x)

#print(Schedule)

In [4]:
#Combine Total Stats
Dataframes = [Week_1_Stats, Week_2_Stats]
Week = len(Dataframes)+1
Combined = pd.concat(Dataframes, ignore_index=True)
Numeric_Part = Combined.groupby('Player', as_index=False).sum(numeric_only=True)
Non_Numeric_Part = Combined.groupby('Player', as_index=False).first(numeric_only=False).drop(columns=Numeric_Part.columns[1:])
Total_Stats = pd.merge(Numeric_Part, Non_Numeric_Part, on='Player')
#print(Total_Stats.columns)

In [5]:
Usefulcolumns = ['Player', 'Team', 'Opp', 'Week', 'Pos.', 'G', 'PassAtt', 'PassAttStDev', 'Cmp', 'IndComp%', 'CompStDev', 'TeamComp%', 'PassYds', 'PassYdsStDev', 'PassYds%', 'PassTD', 'PassTDStDev', 'PassTD%','Tgt', 'TgtStDev','Rec', 'IndCatch%', 'CatStDev', 'TmCatch%', 'RecYds', 'RecYdsStDev', 'RecYds%', 'RecTD', 'RecTDStDev', 'RecTD%', 'RushAtt', 'RushStDev', 'Rush%', 'RushYds', 'RushYdsStDev', 'RushYds%', 'RushTD', 'RushTDStDev', 'RushTD%', ]
Useful = pd.DataFrame(columns=Usefulcolumns)
Opponentcolumns = ['CompAAV', 'PassYdsAAV', 'PassTDAAV', 'CatchAAV', 'RecYdsAAV', 'RecTDAAV', 'RushAAV', 'RushYdsAAV', 'RushTDAAV']
Opponents = pd.DataFrame(columns=Opponentcolumns)
#print(Useful)
#print(Opponents)

In [6]:
#Populate with total stats
for col in Useful.columns:
    
    # Updates the Current Week
    Useful['Week'] = Week 

    # Updates the totals
    if col in Total_Stats.columns: 
        Useful[col] = Total_Stats[col]
    
    # Updates Individual Columns
    for i in range(len(Useful)): 
        team = Useful.loc[i, 'Team']
        player = Useful.loc[i, 'Player']
        count = 0
        pass_attempts = []
        completions = []
        pass_yards = []
        pass_td = []
        targets = []
        receptions = []
        rec_yards = []
        rec_td = []
        rush_attempts = []
        rush_yards = []
        rush_td = []
        


        # Updates Opponents
        for j in range(len(Schedule)):
            if Schedule.iloc[j, 0] == team:
                Useful.loc[i, 'Opp'] = Schedule.iloc[j, Week]  
                break 
        
        # Updates more complicated shit
        for df in Dataframes:
            matches = df[df['Player'] == player]

            # Updates Games Played
            if 'Player' in df.columns:
                count+= (df['Player'] == player).sum()

            # Updates PassAttStDev
            if 'Player' in df.columns and 'PassAtt' in df.columns:
                pass_attempts.extend(matches['PassAtt'].dropna().tolist())

            # Updates CmpStDEv
            if 'Player' in df.columns and 'Cmp' in df.columns:
                completions.extend(matches['Cmp'].dropna().tolist())
            
            # Updates PassYdsStDev
            if 'Player' in df.columns and 'PassYds' in df.columns:
                pass_yards.extend(matches['PassYds'].dropna().tolist())
            
            # Updates PassTDStDev
            if 'Player' in df.columns and 'PassTD' in df.columns:
                pass_td.extend(matches['PassTD'].dropna().tolist())

            # Updates TgtStDev
            if 'Player' in df.columns and 'Tgt' in df.columns:
                targets.extend(matches['Tgt'].dropna().tolist())
            
            # Updates CatStDev
            if 'Player' in df.columns and 'Rec' in df.columns:
                receptions.extend(matches['Rec'].dropna().tolist())
            
            # Updates RecYdsStDev
            if 'Player' in df.columns and 'RecYds' in df.columns:
                rec_yards.extend(matches['RecYds'].dropna().tolist())

            # Updates RecTDStDev
            if 'Player' in df.columns and 'RecTD' in df.columns:
                #matches = df[df['Player'] == player]
                rec_td.extend(matches['RecTD'].dropna().tolist())

            # Updates RushStDev
            if 'Player' in df.columns and 'RushAtt' in df.columns:
                rush_attempts.extend(matches['RushAtt'].dropna().tolist())

            # Updates RushYdsStDev
            if 'Player' in df.columns and 'RushYds' in df.columns:
                rush_yards.extend(matches['RushYds'].dropna().tolist())

            # Updates RushTDStDev
            if 'Player' in df.columns and 'RushTD' in df.columns:
                rush_td.extend(matches['RushTD'].dropna().tolist())


        # Does Standard Deviations
        if len(pass_attempts) >= 2:
            Useful.loc[i, 'PassAttStDev'] = np.std(pass_attempts, ddof=1)
        else:
            Useful.loc[i, 'PassAttStDev'] = 0

        if len(completions) >= 2:
            Useful.loc[i, 'CompStDev'] = np.std(completions, ddof=1)
        else:
            Useful.loc[i, 'CompStDev'] = 0
        
        if len(pass_yards) >= 2:
            Useful.loc[i, 'PassYdsStDev'] = np.std(pass_yards, ddof=1)
        else:
            Useful.loc[i, 'PassYdsStDev'] = 0
        
        if len(pass_td) >= 2:
            Useful.loc[i, 'PassTDStDev'] = np.std(pass_td, ddof=1)
        else:
            Useful.loc[i, 'PassTDStDev'] = 0
        
        if len(targets) >= 2:
            Useful.loc[i, 'TgtStDev'] = np.std(targets, ddof=1)
        else:
            Useful.loc[i, 'TgtStDev'] = 0

        if len(receptions) >= 2:
            Useful.loc[i, 'CatStDev'] = np.std(receptions, ddof=1)
        else:
            Useful.loc[i, 'CatStDev'] = 0
        
        if len(rec_yards) >= 2:
            Useful.loc[i, 'RecYdsStDev'] = np.std(rec_yards, ddof=1)
        else:
            Useful.loc[i, 'RecYdsStDev'] = 0

        if len(rec_td) >= 2:
            Useful.loc[i, 'RecTDStDev'] = np.std(rec_td, ddof=1)
        else:
            Useful.loc[i, 'RecTDStDev'] = 0

        if len(rush_attempts) >= 2:
            Useful.loc[i, 'RushStDev'] = np.std(rush_attempts, ddof=1)
        else:
            Useful.loc[i, 'RushStDev'] = 0

        if len(rush_yards) >= 2:
            Useful.loc[i, 'RushYdsStDev'] = np.std(rush_yards, ddof=1)
        else:
            Useful.loc[i, 'RushYdsStDev'] = 0

        if len(rush_td) >= 2:
            Useful.loc[i, 'RushTDStDev'] = np.std(rush_td, ddof=1)
        else:
            Useful.loc[i, 'RushTDStDev'] = 0


        Useful.loc[i, 'G'] = count

# Update Individual Completion Percentage
Useful['IndComp%'] = np.where(
    Useful['PassAtt'] != 0,
    (Useful['Cmp'] / Useful['PassAtt']),
    0
)

# Update Individual Catch Percentage
Useful['IndCatch%'] = np.where(
    Useful['Tgt'] != 0,
    (Useful['Rec'] / Useful['Tgt']),
    0
)



#print(Useful)

In [7]:
# Collect totals ONLY for games the player played in

# Initialize a list to collect results
totals = []
teamtotals = []

# Loop through each player in Useful
for player in Useful['Player']:
    total_team_pass_att = 0  # Total sum across all DataFrames
    total_team_pass_yards = 0
    total_team_pass_td = 0
    total_team_rec = 0
    total_team_rec_yards = 0
    total_team_rec_td = 0
    total_team_rush_att = 0
    total_team_rush_yards = 0
    total_team_rush_td = 0

    for df in Dataframes:
        if player in df['Player'].values:
            # Get the player's team (assumes 1 team per player per df)
            player_team = df.loc[df['Player'] == player, 'Team'].iloc[0]

            # Sum Cmp for all players on the same team
            team_total_PA = df.loc[df['Team'] == player_team, 'PassAtt'].sum()
            team_total_PY = df.loc[df['Team'] == player_team, 'PassYds'].sum()
            team_total_PT = df.loc[df['Team'] == player_team, 'PassTD'].sum()
            team_total_R = df.loc[df['Team'] == player_team, 'Rec'].sum()
            team_total_RY = df.loc[df['Team'] == player_team, 'RecYds'].sum()
            team_total_RT = df.loc[df['Team'] == player_team, 'RecTD'].sum()
            team_total_Ru = df.loc[df['Team'] == player_team, 'RushAtt'].sum()
            team_total_RuY = df.loc[df['Team'] == player_team, 'RushYds'].sum()
            team_total_RuT = df.loc[df['Team'] == player_team, 'RushTD'].sum()

            total_team_pass_att += team_total_PA
            total_team_pass_yards += team_total_PY
            total_team_pass_td += team_total_PT
            total_team_rec += team_total_R
            total_team_rec_yards += team_total_RY
            total_team_rec_td += team_total_RT
            total_team_rush_att += team_total_Ru
            total_team_rush_yards += team_total_RuY
            total_team_rush_td += team_total_RuT

    # Save result
    totals.append({'Player': player, 'TeamTotalPassAtt': total_team_pass_att, 'TeamTotalPassYds': total_team_pass_yards, 'TeamTotalPassTD': total_team_pass_td, 'TeamTotalRec': total_team_rec, 'TeamTotalRecYds': total_team_rec_yards, 'TeamTotalRecTD': total_team_rec_td, 'TeamTotalRushAtt': total_team_rush_att, 'TeamTotalRushYds': total_team_rush_yards, 'TeamTotalRushTD': total_team_rush_td})

# Create the final result DataFrame
IndividualTotals = pd.DataFrame(totals)
#print(totals)
#print(IndividualTotals)

In [8]:
# Update Team Comp Percentage
Useful['TeamComp%'] = np.where(
    IndividualTotals['TeamTotalPassAtt'] != 0,
    (Useful['Cmp'] / IndividualTotals['TeamTotalPassAtt']),
    0
)

# Update Team Pass Yards Percentage
Useful['PassYds%'] = np.where(
    IndividualTotals['TeamTotalPassYds'] != 0,
    (Useful['PassYds'] / IndividualTotals['TeamTotalPassYds']),
    0
)

# Update Team Pass TD Percentage
Useful['PassTD%'] = np.where(
    IndividualTotals['TeamTotalPassTD'] != 0,
    (Useful['PassTD'] / IndividualTotals['TeamTotalPassTD']),
    0
)

# Update Team Catch Percentage
Useful['TmCatch%'] = np.where(
    IndividualTotals['TeamTotalRec'] != 0,
    (Useful['Rec'] / IndividualTotals['TeamTotalRec']),
    0
)

# Update Team Receiving Yards Percentage
Useful['RecYds%'] = np.where(
    IndividualTotals['TeamTotalRecYds'] != 0,
    (Useful['RecYds'] / IndividualTotals['TeamTotalRecYds']),
    0
)

# Update Team Receiving TD Percentage
Useful['RecTD%'] = np.where(
    IndividualTotals['TeamTotalRecTD'] != 0,
    (Useful['RecTD'] / IndividualTotals['TeamTotalRecTD']),
    0
)

# Update Team Rush Percentage
Useful['Rush%'] = np.where(
    IndividualTotals['TeamTotalRushAtt'] != 0,
    (Useful['RushAtt'] / IndividualTotals['TeamTotalRushAtt']),
    0
)

# Update Team Rush Yard Percentage
Useful['RushYds%'] = np.where(
    IndividualTotals['TeamTotalRushYds'] != 0,
    (Useful['RushYds'] / IndividualTotals['TeamTotalRushYds']),
    0
)

# Update Team Rush TD Percentage
Useful['RushTD%'] = np.where(
    IndividualTotals['TeamTotalRushTD'] != 0,
    (Useful['RushTD'] / IndividualTotals['TeamTotalRushTD']),
    0
)

#print(Useful[:2])

In [9]:

    # Initialize a list to collect results
teamtotals = []

# Loop through each player in Useful
for team in Schedule['Team']:
    total_team_completions = 0  # Total sum across all DataFrames
    total_team_pass_yards = 0
    total_team_pass_td = 0
    total_team_rec = 0
    total_team_rec_yards = 0
    total_team_rec_td = 0
    total_team_rush_att = 0
    total_team_rush_yards = 0
    total_team_rush_td = 0

    for df in Dataframes:
        #if player in df['Player'].values:
            # Get the player's team (assumes 1 team per player per df)
        #team = df.loc[df['Team'] == team, 'Team'].iloc[0]

        # Sum Cmp for all players on the same team
        team_total_C = df.loc[df['Team'] == team, 'Cmp'].sum()
        team_total_PY = df.loc[df['Team'] == team, 'PassYds'].sum()
        team_total_PT = df.loc[df['Team'] == team, 'PassTD'].sum()
        team_total_R = df.loc[df['Team'] == team, 'Rec'].sum()
        team_total_RY = df.loc[df['Team'] == team, 'RecYds'].sum()
        team_total_RT = df.loc[df['Team'] == team, 'RecTD'].sum()
        team_total_Ru = df.loc[df['Team'] == team, 'RushAtt'].sum()
        team_total_RuY = df.loc[df['Team'] == team, 'RushYds'].sum()
        team_total_RuT = df.loc[df['Team'] == team, 'RushTD'].sum()

        total_team_completions += team_total_C
        total_team_pass_yards += team_total_PY
        total_team_pass_td += team_total_PT
        total_team_rec += team_total_R
        total_team_rec_yards += team_total_RY
        total_team_rec_td += team_total_RT
        total_team_rush_att += team_total_Ru
        total_team_rush_yards += team_total_RuY
        total_team_rush_td += team_total_RuT

    # Save result
    teamtotals.append({'Team': team, 'CmpAAV': total_team_completions, 'PassYdsAAV': total_team_pass_yards, 'PassTDAAV': total_team_pass_td, 'RecAAV': total_team_rec, 'RecYdsAAV': total_team_rec_yards, 'RecTDAAV': total_team_rec_td, 'RushAttAAV': total_team_rush_att, 'RushYdsAAV': total_team_rush_yards, 'RushTDAAV': total_team_rush_td})

TeamTotals = pd.DataFrame(teamtotals)

for column in TeamTotals.columns[1:]:
    row_index=0
    
    if (df.iloc[row_index, :len(Dataframes)] == 'BYE').any():
        TeamTotals[column] = (TeamTotals[column] - TeamTotals[column].mean())/(len(Dataframes-1))
    else:
        TeamTotals[column] = (TeamTotals[column] - TeamTotals[column].mean())/len(Dataframes)
#print(TeamTotals.columns)


In [10]:
SuperFlexcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
SuperFlex = pd.DataFrame(columns=SuperFlexcolumns)
Flexcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
Flex = pd.DataFrame(columns=Flexcolumns)
WRcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
WR = pd.DataFrame(columns=WRcolumns)
RBcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
RB = pd.DataFrame(columns=RBcolumns)
TEcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
TE = pd.DataFrame(columns=TEcolumns)
QBcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
QB = pd.DataFrame(columns=QBcolumns)

#Populate Superflex with Player names
SuperFlex['Player'] = Useful['Player']

#Populate Flex with Player names
for i, row in Useful.iterrows():
    keywords = ['WR', 'RB', 'TE']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        Flex.at[i, 'Player'] = row['Player']

#Populate WR with Player names
for i, row in Useful.iterrows():
    keywords = ['WR']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        WR.at[i, 'Player'] = row['Player']

#Populate RB with Player names
for i, row in Useful.iterrows():
    keywords = ['RB']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        RB.at[i, 'Player'] = row['Player']

#Populate TE with Player names
for i, row in Useful.iterrows():
    keywords = ['TE']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        TE.at[i, 'Player'] = row['Player']

#Populate QB with Player names
for i, row in Useful.iterrows():
    keywords = ['QB']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        QB.at[i, 'Player'] = row['Player']




In [13]:
run=0
MonteCarlorushes = []
MonteCarlorushyards = []
MonteCarlorushtds = []
MonteCarloreceptions = []
MonteCarloreceivingyards = []
MonteCarloreceivingtds = []
MonteCarlopassingyards = []
MonteCarlopassingtds = []

while run<10000:
    for player in Useful['Player']:

        expectedrushes = (Useful['RushAtt']/Useful['G']) + Useful['RushStDev']*np.random.uniform(-2,2) + (TeamTotals).loc[TeamTotals['Team'] == Useful.loc[0, 'Opp'], 'RushAttAAV'].iloc[0]*Useful['Rush%']
        expectedrushyards = (Useful['RushYds']/Useful['G']) + Useful['RushYdsStDev']*np.random.uniform(-2,2) + (TeamTotals).loc[TeamTotals['Team'] == Useful.loc[0, 'Opp'], 'RushYdsAAV'].iloc[0]*Useful['RushYds%']
        expectedrushtds = (Useful['RushTD']/Useful['G']) + Useful['RushTDStDev']*np.random.uniform(-2,2) + (TeamTotals).loc[TeamTotals['Team'] == Useful.loc[0, 'Opp'], 'RushTDAAV'].iloc[0]*Useful['RushTD%']
        MonteCarlorushes.append(expectedrushes)
        MonteCarlorushyards.append(expectedrushyards)
        MonteCarlorushtds.append(expectedrushtds)
        expectedreceptions = (Useful['Tgt']/Useful['G'])*Useful['IndCatch%'] + Useful['TgtStDev']*Useful['IndCatch%']*np.random.uniform(-2,2) + (TeamTotals).loc[TeamTotals['Team'] == Useful.loc[0, 'Opp'], 'RecAAV'].iloc[0]*Useful['TmCatch%']
        expectedreceivingyards = (Useful['RecYds']/Useful['G']) + Useful['RecYdsStDev']*np.random.uniform(-2,2) + (TeamTotals).loc[TeamTotals['Team'] == Useful.loc[0, 'Opp'], 'RecYdsAAV'].iloc[0]*Useful['RecYds%']
        expectedreceivingtds = (Useful['RecTD']/Useful['G']) + Useful['RecTDStDev']*np.random.uniform(-2,2) + (TeamTotals).loc[TeamTotals['Team'] == Useful.loc[0, 'Opp'], 'RecTDAAV'].iloc[0]*Useful['RecTD%']
        MonteCarloreceptions.append(expectedreceptions)
        MonteCarloreceivingyards.append(expectedreceivingyards)
        MonteCarloreceivingtds.append(expectedreceivingtds)
        expectedpassingyards = (Useful['PassYds']/Useful['G']) + Useful['PassYdsStDev']*np.random.uniform(-2,2) + (TeamTotals).loc[TeamTotals['Team'] == Useful.loc[0, 'Opp'], 'PassYdsAAV'].iloc[0]*Useful['PassYds%']
        expectedpassingtds = (Useful['PassTD']/Useful['G']) + Useful['PassTDStDev']*np.random.uniform(-2,2) + (TeamTotals).loc[TeamTotals['Team'] == Useful.loc[0, 'Opp'], 'PassTDAAV'].iloc[0]*Useful['PassTD%']
        MonteCarlopassingyards.append(expectedpassingyards)
        MonteCarlopassingtds.append(expectedpassingtds)
        run = run+1
        predictedrushes = sum(MonteCarlorushes)/len(MonteCarlorushes)
        predictedrushyards = sum(MonteCarlorushyards)/len(MonteCarlorushyards)
        predictedrushtds = sum(MonteCarlorushtds)/len(MonteCarlorushtds)
        predictedreceptions = sum(MonteCarloreceptions)/len(MonteCarloreceptions)
        predictedreceivingyards = sum(MonteCarloreceivingyards)/len(MonteCarloreceivingyards)
        predictedreceivingtds = sum(MonteCarloreceivingtds)/len(MonteCarloreceivingtds)
        predictedpassingyards = sum(MonteCarlopassingyards)/len(MonteCarlopassingyards)
        predictedpassingtds = sum(MonteCarlopassingtds)/len(MonteCarlopassingtds)
        ppr = (predictedrushyards + predictedreceivingyards)/10 + predictedpassingyards/25 + predictedreceptions + (predictedrushtds + predictedreceivingtds)*6 + predictedpassingtds*4
        std = (predictedrushyards + predictedreceivingyards)/10 + predictedpassingyards/25 + (predictedrushtds + predictedreceivingtds)*6 + predictedpassingtds*4

SuperFlex['RushAtt'] = predictedrushes
SuperFlex['RushYds'] = predictedrushyards
SuperFlex['RushTD'] = predictedrushtds
SuperFlex['Rec'] = predictedreceptions
SuperFlex['RecYds'] = predictedreceivingyards
SuperFlex['RecTD'] = predictedreceivingtds
SuperFlex['PassYds'] = predictedpassingyards
SuperFlex['PassTD'] = predictedpassingtds
SuperFlex['PPR'] = ppr
SuperFlex['STD'] = std
SuperFlex.iloc[:, 1:3] = SuperFlex.iloc[:, 1:3].apply(pd.to_numeric).round(1)
SuperFlex.iloc[:, 3:] = SuperFlex.iloc[:, 3:].apply(pd.to_numeric).round(0).astype('Int64')

for col in SuperFlex.columns:
    if col != 'Player' and col in Flex.columns:
        Flex[col] = Flex['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in WR.columns:
        WR[col] = WR['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in RB.columns:
        RB[col] = RB['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in TE.columns:
        TE[col] = TE['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in QB.columns:
        QB[col] = QB['Player'].map(SuperFlex.set_index('Player')[col])


#print(QB[:10])


KeyboardInterrupt: 

In [12]:
print(QB[:10])

            Player  PPR  STD PassYds PassTD   Rec RecYds RecTD RushAtt  \
4    Aaron Rodgers  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
25  Baker Mayfield  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
35          Bo Nix  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
48     Brock Purdy  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
51     Bryce Young  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
54     C.J. Stroud  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
57  Caleb Williams  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
61        Cam Ward  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
87     Cooper Rush  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   
95    Dak Prescott  NaN  NaN    <NA>   <NA>  <NA>   <NA>  <NA>    <NA>   

   RushYds RushTD  
4     <NA>   <NA>  
25    <NA>   <NA>  
35    <NA>   <NA>  
48    <NA>   <NA>  
51    <NA>   <NA>  
54    <NA>   <NA>  
57    <NA>   <NA>  
61    <NA>   <NA>  
87   

In [23]:

n_simulations = 10000
team_stats = TeamTotals.set_index('Team').to_dict('index')

players = []

predictedrushes = []
predictedrushyards = []
predictedrushtds = []

predictedreceptions = []
predictedreceivingyards = []
predictedreceivingtds = []

predictedpassingyards = []
predictedpassingtds = []

pprs = []
stds = []

for i, row in Useful.iterrows():
    opp = row['Opp']
    if opp not in team_stats:
        continue

    team = team_stats[opp]
    rand = np.random.uniform(-2, 2, n_simulations)

    # Simulations
    rushes = (row['RushAtt'] / row['G']) + row['RushStDev'] * rand + team['RushAttAAV'] * row['Rush%']
    rushyards = (row['RushYds'] / row['G']) + row['RushYdsStDev'] * rand + team['RushYdsAAV'] * row['RushYds%']
    rushtds = (row['RushTD'] / row['G']) + row['RushTDStDev'] * rand + team['RushTDAAV'] * row['RushTD%']

    receptions = (row['Tgt'] / row['G']) * row['IndCatch%'] + row['TgtStDev'] * row['IndCatch%'] * rand + team['RecAAV'] * row['TmCatch%']
    receivingyards = (row['RecYds'] / row['G']) + row['RecYdsStDev'] * rand + team['RecYdsAAV'] * row['RecYds%']
    receivingtds = (row['RecTD'] / row['G']) + row['RecTDStDev'] * rand + team['RecTDAAV'] * row['RecTD%']

    passingyards = (row['PassYds'] / row['G']) + row['PassYdsStDev'] * rand + team['PassYdsAAV'] * row['PassYds%']
    passingtds = (row['PassTD'] / row['G']) + row['PassTDStDev'] * rand + team['PassTDAAV'] * row['PassTD%']

    players.append(row['Player'])

    predictedrushes.append(np.round(rushes.mean()).astype(int))
    predictedrushyards.append(np.round(rushyards.mean()).astype(int))
    predictedrushtds.append(np.round(rushtds.mean(),1))

    predictedreceptions.append(np.round(receptions.mean()).astype(int))
    predictedreceivingyards.append(np.round(receivingyards.mean()).astype(int))
    predictedreceivingtds.append(np.round(receivingtds.mean(),1))

    predictedpassingyards.append(np.round(passingyards.mean()).astype(int))
    predictedpassingtds.append(np.round(passingtds.mean(),1))
    

    # Fantasy scoring
    ppr = (
        np.round(rushyards.mean()).astype(int) / 10 +
        np.round(receivingyards.mean()).astype(int) / 10 +
        np.round(passingyards.mean()).astype(int) / 25 +
        np.round(receptions.mean()).astype(int) +
        (np.round(rushtds.mean(),1) + np.round(receivingtds.mean(),1)) * 6 +
        np.round(passingtds.mean(),1) * 4
    )
    pprs.append(ppr)

    std = (
        np.round(rushyards.mean()).astype(int) / 10 +
        np.round(receivingyards.mean()).astype(int) / 10 +
        np.round(passingyards.mean()).astype(int) / 25 +
        (np.round(rushtds.mean(),1) + np.round(receivingtds.mean(),1)) * 6 +
        np.round(passingtds.mean(),1) * 4
    )
    stds.append(std)

SuperFlex['RushAtt'] = predictedrushes
SuperFlex['RushYds'] = predictedrushyards
SuperFlex['RushTD'] = predictedrushtds
SuperFlex['Rec'] = predictedreceptions
SuperFlex['RecYds'] = predictedreceivingyards
SuperFlex['RecTD'] = predictedreceivingtds
SuperFlex['PassYds'] = predictedpassingyards
SuperFlex['PassTD'] = predictedpassingtds
SuperFlex['PPR'] = pprs
SuperFlex['STD'] = stds
SuperFlex.iloc[:, 1:3] = SuperFlex.iloc[:, 1:3].apply(pd.to_numeric).round(1)

for col in SuperFlex.columns:
    if col != 'Player' and col in Flex.columns:
        Flex[col] = Flex['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in WR.columns:
        WR[col] = WR['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in RB.columns:
        RB[col] = RB['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in TE.columns:
        TE[col] = TE['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in QB.columns:
        QB[col] = QB['Player'].map(SuperFlex.set_index('Player')[col])

In [24]:
print(SuperFlex[:10])

            Player   PPR   STD  PassYds  PassTD  Rec  RecYds  RecTD  RushAtt  \
0       A.J. Brown   5.3   2.3        0     0.0    3      23    0.0        0   
1        AJ Barner   6.4   4.4        0     0.0    2      12    0.5        1   
2        AJ Dillon   1.4   1.4        0     0.0    0       0    0.0        3   
3      Aaron Jones   8.0   7.0        0     0.0    1      20    0.7        5   
4    Aaron Rodgers  20.2  20.2      253     2.5    0       0    0.0        1   
5    Adam Prentice   1.3   0.3        0     0.0    1       3    0.0        0   
6     Adam Thielen   2.2   1.2        0     0.0    1      12    0.0        0   
7    Adam Trautman   7.8   5.8        0     0.0    2      10    0.8        0   
8  Adonai Mitchell   2.5   1.5        0     0.0    1      15    0.0        0   
9      Alec Ingold   2.9   0.9        0     0.0    2       9    0.0        0   

   RushYds  RushTD  
0        0     0.0  
1        2     0.0  
2       14     0.0  
3        8     0.0  
4        1    